In [242]:
import json
import os
import sys

import boto3
import asyncio
from typing import List

from langchain_community.llms import Bedrock
from langchain_community.chat_models import BedrockChat

from langchain.prompts import ChatPromptTemplate, PromptTemplate

from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
#from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import GithubFileLoader
from langchain_core.runnables import RunnablePassthrough


model_id = "mistral.mistral-large-2402-v1:0"

model_kwargs =  { 
    "max_tokens": 6000
}

bedrock_runtime = boto3.client('bedrock-runtime')

llm = BedrockChat(
    client=bedrock_runtime,
    model_id=model_id,
)

chat = BedrockChat(model_id=model_id)

In [48]:
! pip install asyncio
# parser = StrOutputParser()

In [57]:
response_schemas = [
    ResponseSchema(name="Output", description="modernize and generate the code in java format"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [154]:
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="You are a software architect, answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

#print (format_instructions)

In [111]:
# If you haven't set your access token as an environment variable, pass it in here.
from getpass import getpass

ACCESS_TOKEN = getpass()


 ········


In [124]:
loader = GithubFileLoader(
    repo="jihed/helloworld-mdb",
    access_token=ACCESS_TOKEN,  # delete/comment out this argument if you've set the access token as an env var.
    github_api_url="https://api.github.com",
    file_filter=lambda file_path: file_path.endswith(
        ".java"
    ),  # load all markdowns files.
)

In [130]:
documents = loader.load()

In [151]:
for document in documents:
    print(document.metadata)


{'path': 'src/main/java/org/jboss/as/quickstarts/mdb/HelloWorldQueueMDB.java', 'sha': '560f22a0aafcda739097cb38e8ddc787a8c2e9ce', 'source': 'https://api.github.com/jihed/helloworld-mdb/blob/main/src/main/java/org/jboss/as/quickstarts/mdb/HelloWorldQueueMDB.java'}
{'path': 'src/main/java/org/jboss/as/quickstarts/mdb/HelloWorldTopicMDB.java', 'sha': 'acd7de34f25785bb5c8d9a8cb9068e71ea9e0a4b', 'source': 'https://api.github.com/jihed/helloworld-mdb/blob/main/src/main/java/org/jboss/as/quickstarts/mdb/HelloWorldTopicMDB.java'}
{'path': 'src/main/java/org/jboss/as/quickstarts/servlet/HelloWorldMDBServletClient.java', 'sha': '095d3097a8e6f6edbd500e991ddac71ef0914ff8', 'source': 'https://api.github.com/jihed/helloworld-mdb/blob/main/src/main/java/org/jboss/as/quickstarts/servlet/HelloWorldMDBServletClient.java'}


In [214]:
!pip install -qU langchain-text-splitters

In [224]:
# Full list of supported languages
[e.value for e in Language]

['cpp',
 'go',
 'java',
 'kotlin',
 'js',
 'ts',
 'php',
 'proto',
 'python',
 'rst',
 'ruby',
 'rust',
 'scala',
 'swift',
 'markdown',
 'latex',
 'html',
 'sol',
 'csharp',
 'cobol',
 'c',
 'lua',
 'perl']

In [226]:
RecursiveCharacterTextSplitter.get_separators_for_language(Language.JAVA)

['\nclass ',
 '\npublic ',
 '\nprotected ',
 '\nprivate ',
 '\nstatic ',
 '\nif ',
 '\nfor ',
 '\nwhile ',
 '\nswitch ',
 '\ncase ',
 '\n\n',
 '\n',
 ' ',
 '']

In [319]:
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_messages(
    [("system", "Generate for me a step by step guide in mardown format to migrate all the documents in context written in enterprise Java EE source code to Quarkus:\n\n{context}")]
)
chain = create_stuff_documents_chain(llm, prompt)
document = Document(page_content=documents[0].page_content)

response = chain.invoke({"context": documents})

print(response, end= '\n')



# Migrating from Enterprise Java EE to Quarkus

Quarkus is a Kubernetes Native Java stack tailored for OpenJDK HotSpot and GraalVM, crafted from the best of breed Java libraries and standards. Here is a step-by-step guide to migrate your Enterprise Java EE application to Quarkus.

## Prerequisites

- Basic understanding of Java and Java EE concepts
- Java Development Kit (JDK) installed on your system
- Apache Maven or Gradle build tool installed on your system
- Quarkus CLI installed on your system

## Step 1: Create a new Quarkus project

First, you need to create a new Quarkus project. You can do this using the Quarkus CLI or your build tool.

Using Quarkus CLI:

```bash
quarkus create app --name=my-quarkus-app
```

Using Maven:

```bash
mvn io.quarkus:quarkus-maven-plugin:1.13.3.Final:create \
    -DprojectGroupId=org.acme \
    -DprojectArtifactId=my-quarkus-app \
    -DclassName="org.acme.QuickstartResource" \
    -Dpath="/hello"
```

Using Gradle:

```bash
./gradlew createApp 

In [259]:
from langchain.chains import RefineDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate

# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate(
    input_variables=["page_content"],
     template="{page_content}"
)
document_variable_name = "context"

# The prompt here should take as an input variable the
# `document_variable_name`
prompt = PromptTemplate.from_template(
    "Migrate this content: {context} from Java Entreprise to Quarkus"
)
initial_llm_chain = LLMChain(llm=llm, prompt=prompt)
initial_response_name = "prev_response"
# The prompt here should take as an input variable the
# `document_variable_name` as well as `initial_response_name`
prompt_refine = PromptTemplate.from_template(
    "Here's your Migration Plan: {prev_response}. "
    "Now generate a quarkus code for: {context}"
)
refine_llm_chain = LLMChain(llm=llm, prompt=prompt_refine)
chain = RefineDocumentsChain(
    initial_llm_chain=initial_llm_chain,
    refine_llm_chain=refine_llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
    initial_response_name=initial_response_name,
)



response = chain.invoke(documents)



In [306]:
print(response.get())

TypeError: get expected at least 1 argument, got 0

In [313]:
keys = response.keys()
print (keys)

dict_keys(['input_documents', 'output_text'])


In [318]:
print (response.get('input_documents'), end= '\n')

[Document(page_content='/*\n * JBoss, Home of Professional Open Source\n * Copyright 2015, Red Hat, Inc. and/or its affiliates, and individual\n * contributors by the @authors tag. See the copyright.txt in the\n * distribution for a full listing of individual contributors.\n *\n * Licensed under the Apache License, Version 2.0 (the "License");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n * http://www.apache.org/licenses/LICENSE-2.0\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an "AS IS" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\npackage org.jboss.as.quickstarts.mdb;\n\nimport java.util.logging.Logger;\nimport javax.ejb.ActivationConfigProperty;\nimport javax.ejb.MessageDriven;\nimport jav

In [315]:
print (response.get('output_text'), end= '\n')

 To migrate the given Java EE Servlet to Quarkus, you would need to use the Quarkus JMS and RESTEasy frameworks. Here's how you can migrate the code:

```java
import io.quarkus.jms.JMSDestination;
import io.quarkus.jms.JMSProducer;
import io.quarkus.qute.Template;
import io.quarkus.qute.TemplateInstance;
import io.quarkus.qute.Location;
import javax.inject.Inject;
import javax.jms.Queue;
import javax.jms.Topic;
import javax.ws.rs.GET;
import javax.ws.rs.Path;
import javax.ws.rs.Produces;
import javax.ws.rs.QueryParam;
import javax.ws.rs.core.MediaType;
import java.util.List;

@Path("/hello")
public class HelloWorldMDBServletClient {

    private static final int MSG_COUNT = 5;

    @Inject
    @JMSProducer
    javax.jms.JMSContext context;

    @Inject
    @JMSDestination("java:/queue/HELLOWORLDMDBQueue")
    Queue queue;

    @Inject
    @JMSDestination("java:/topic/HELLOWORLDMDBTopic")
    Topic topic;

    @Location("hello.txt")
    Template hello;

    @GET
    @Produces(MediaType.

In [327]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

python_code = """
def hello_world():
    print("Hello, World!")
hello_world()
"""

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
)
python_docs = python_splitter.create_documents([python_code])
print(python_docs[0])

page_content='def hello_world():\n    print("Hello, World!")'


In [344]:
sample = documents[0].page_content

java_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.JAVA, chunk_size=1000, chunk_overlap=1000
)
sample_content = splitter.split_documents(documents)
#print(sample_content)